## 3.1. Preprocessor for SIFT feature target generator

In [1]:
import time

import numpy as np
import tensorflow as tf
import tensorpack as tp

import cv2
import matplotlib.pyplot as plt
%matplotlib inline

In [2]:
from tensorpack.dataflow import *

class SampleList(RNGDataFlow):
    def __init__(self, shuffle=False):
        self.filelist = [
            ('datas/yosemite_test/yosemite1.jpg', 1),
            ('datas/yosemite_test/yosemite2.jpg', 2),
            ('datas/yosemite_test/yosemite3.jpg', 3),
            ('datas/yosemite_test/yosemite4.jpg', 4),
        ]
        self.shuffle = shuffle
        self.detector = cv2.xfeatures2d.SIFT_create()
        
    def size(self):
        return len(self.filelist)
    
    def preprocessing(self, filename):
        image = cv2.imread(filename)
        shape = image.shape[:2]
        
        keypoints = self.detector.detect(cv2.cvtColor(cv2.imread(filename), cv2.COLOR_BGR2GRAY))
        feature = np.zeros( shape + (3,) )
        for kp in keypoints:
            x = int(kp.pt[0]+0.5)
            y = int(kp.pt[1]+0.5)
            feature[y][x][0] = kp.response
            feature[y][x][1] = kp.size / 100.0
            feature[y][x][2] = kp.angle / 360.0
        return [image, feature]
        
    def get_data(self):
        idxs = np.arange(len(self.filelist))
        if self.shuffle:
            self.rng.shuffle(idxs)
        for k in idxs:
            filename, label = self.filelist[k]
            yield self.preprocessing(filename)

ds = SampleList()
ds = BatchData(ds, 2, use_list=False)
ds = PrintData(ds, num=2)
TestDataSpeed(ds, size=2).start()

  0%|          |0/2[00:00<?,?it/s]

[1105 00:59:52 @common.py:726] DataFlow Info:
datapoint 0<2 with 2 components consists of
   dp 0: is ndarray of shape (2, 480, 640, 3) with range [3.0000, 255.0000]
   dp 1: is ndarray of shape (2, 480, 640, 3) with range [0.0000, 2.0002]


100%|##########|2/2[00:00<00:00, 3.90it/s]

datapoint 1<2 with 2 components consists of
   dp 0: is ndarray of shape (2, 480, 640, 3) with range [0.0000, 255.0000]
   dp 1: is ndarray of shape (2, 480, 640, 3) with range [0.0000, 1.2752]
